## Deliverable
- modified ISO.csv with the new joined columns is the priority for now
- output a column like n_utilities_in_county

## To Do
- Look at how many projects have NAs for county_id_fips - 314 projects of different statuses
- Find out how many utilities are in a county (many utilities operate in one county)
- Join on FIPS code

## Next Steps
- Can try and fill in some of the missing county_id_fips based on county and state (if county and state aren't NA)
- Make a guess based on year
    - create a "cleaned_year" column
    - join based on year and county 

In [26]:
import pandas as pd
import sqlalchemy as sa

### Loading and looking at the Active ISO Projects data

In [65]:
iso_df = pd.read_csv('active_iso_projects.csv', dtype={'state_id_fips': str, 'county_id_fips': str})

In [66]:
iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,date_operational,days_in_queue,queue_date_raw,year_operational,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,500.0000
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,500.0000
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,500.0000
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,725.0000
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wind,4.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,NaN,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offshore Wind,1200.0000
6350,5285,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,6.0000
6351,5352,NaN,NaN,NaN,NaN,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,0.0000
6352,5378,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,0.0055


In [73]:
nan_iso_df = iso_df[iso_df['county_id_fips'].isnull()]
nan_iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,days_in_queue,queue_date_raw,year_operational,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county
50,44,winfield,KS,20,NaN,6/1/2023,NaN,SPP,In Progress,DISIS STAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gas,58.0000,NaN
62,53,elida,NM,35,NaN,12/1/2023,NaN,SPP,In Progress,DISIS STAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,306.0000,NaN
80,70,northhampton,NC,37,NaN,12/31/2017,NaN,PJM,In Progress,In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,180.0000,NaN
81,71,montrail,ND,38,NaN,NaN,NaN,SPP,In Progress,DISIS STAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,74.0000,NaN
94,86,chatauqua,NY,36,NaN,11/2022,SED NY Holdings LLC,NYISO,In Progress,SRIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,20.0000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,NaN,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offshore Wind,1200.0000,NaN
6350,5285,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,6.0000,NaN
6351,5352,NaN,NaN,NaN,NaN,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,0.0000,NaN
6352,5378,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,0.0055,NaN


In [38]:
nan_iso_df['interconnection_status_lbnl'].unique()

array(['In Progress', 'unknown', 'Not Started', 'IA Executed'],
      dtype=object)

In [39]:
iso_df['interconnection_status_lbnl'].unique()

array(['In Progress', 'Not Started', 'IA Executed', 'unknown'],
      dtype=object)

### Load EIA 861 service territories data

In [41]:
import pudl

In [40]:
PUDL_DB_PATH = "/Users/katielamb/Documents/Catalyst_Coop/workspace/sqlite/pudl.sqlite"
pudl_engine = sa.create_engine(f"sqlite:///{PUDL_DB_PATH}")

In [42]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)

In [43]:
serv_terr_df = pudl_out.service_territory_eia861()
serv_terr_df

/Users/katielamb/Documents/Catalyst_Coop/pudl/src/pudl/extract/eia861.py:37: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
The following reported NERC regions are not currently recognized and become         UNK values: []
The following reported NE

,county,short_form,state,utility_id_eia,utility_name_eia,report_date,state_id_fips,county_id_fips
0,Carroll,<NA>,IA,11581,City of Manning,2001-01-01,19,19027
1,Wright,<NA>,MO,11585,City of Mansfield,2001-01-01,29,29229
2,Bristol,<NA>,MA,11586,Town of Mansfield,2001-01-01,25,25005
3,Newton,<NA>,GA,11587,City of Mansfield,2001-01-01,13,13217
4,Sanpete,<NA>,UT,11588,City of Manti,2001-01-01,49,49039
...,...,...,...,...,...,...,...,...
235986,Neosho,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20133
235987,Osage,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20139
235988,Wabaunsee,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20197
235989,Wilson,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20205


In [44]:
len(serv_terr_df['utility_id_eia'].unique())

3061

In [83]:
ids_per_county = serv_terr_df.groupby('county_id_fips')['utility_id_eia'].unique()
ids_per_county = ids_per_county.rename('utility_ids_in_county_eia')
ids_per_county

county_id_fips
01001                       [195, 3222, 30517]
01003    [6491, 6145, 1149, 16140, 17646, 195]
01005                [14602, 18203, 5204, 195]
01007                              [3222, 195]
01009                                    [195]
                         ...                  
72145                                  [15497]
72147                                  [15497]
72149                                  [15497]
72151                                  [15497]
72153                                  [15497]
Name: utility_ids_in_county_eia, Length: 3220, dtype: object

In [64]:
utilities_per_county = ids_per_county.str.len()
utilities_per_county

county_id_fips
01001    3
01003    6
01005    4
01007    2
01009    1
        ..
72145    1
72147    1
72149    1
72151    1
72153    1
Name: utility_id_eia, Length: 3220, dtype: int64

In [68]:
utilities_per_county = utilities_per_county.rename('n_utilities_in_county')

### Join dataframes

In [71]:
iso_df = iso_df.join(utilities_per_county, on="county_id_fips")
iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,days_in_queue,queue_date_raw,year_operational,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,725.0000,8.0
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wind,4.5000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,NaN,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offshore Wind,1200.0000,NaN
6350,5285,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,6.0000,NaN
6351,5352,NaN,NaN,NaN,NaN,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Battery,0.0000,NaN
6352,5378,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Solar,0.0055,NaN


In [77]:
utility_names_per_county = serv_terr_df.groupby('county_id_fips')['utility_name_eia'].unique()
utility_names_per_county = utility_names_per_county.rename('utilities_in_county_eia')

In [78]:
iso_df = iso_df.join(utility_names_per_county, on='county_id_fips')
iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,year_operational,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utility_names_eia,utilities_in_county_eia
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[Yellowstone Valley Elec Co-op, Mid-Yellowston..."
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[Yellowstone Valley Elec Co-op, Mid-Yellowston..."
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[Yellowstone Valley Elec Co-op, Mid-Yellowston..."
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,Battery,725.0000,8.0,"[Imperial Irrigation District, City of Riversi...","[Imperial Irrigation District, City of Riversi..."
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,NaN,Wind,4.5000,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[Kay Electric Cooperative, Central Rural Elect..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,NaN,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,Offshore Wind,1200.0000,NaN,NaN,NaN
6350,5285,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,Solar,6.0000,NaN,NaN,NaN
6351,5352,NaN,NaN,NaN,NaN,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,NaN,NaN,Battery,0.0000,NaN,NaN,NaN
6352,5378,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,Solar,0.0055,NaN,NaN,NaN


In [84]:
iso_df = iso_df.join(ids_per_county, on='county_id_fips')

In [85]:
iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,year_operational,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]"
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]"
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]"
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,Battery,725.0000,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2..."
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,NaN,Wind,4.5000,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,NaN,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,Offshore Wind,1200.0000,NaN,NaN,NaN
6350,5285,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,Solar,6.0000,NaN,NaN,NaN
6351,5352,NaN,NaN,NaN,NaN,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,NaN,NaN,Battery,0.0000,NaN,NaN,NaN
6352,5378,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,Solar,0.0055,NaN,NaN,NaN


In [220]:
iso_df['n_utilities_in_county'].dropna().mean()

4.737086092715232

### Join utility based on county and year

In [88]:
iso_df['date_proposed_raw'].str[-4:].unique()

array(['2023', '2024', '2021', nan, '2019', '2022', '2020', '2018',
       '2012', '2017', '2025', '2015', '4866', '2027', '1899', '2016',
       '5092', '5017', '2026', '2009', '3862', '4835', '2010', '2028',
       '6203', '2011', '2006', '4713', '4986', '4743', '5047', '4423',
       '5051', '4805', 'I/S', '5148', '2014', '2013', '4958', '5139',
       '5261', '4788', '5108', '2033', '4682', '5016', '4671', '4836',
       '4941', '5168', '2031', '4018', '4909', '5078', '5170', '3983',
       '4880', '4742', '2029', '5473', '4864', '5494', '6142', '5495',
       '5046', '4769', '4849', '1/20', '1/21', '1/23', '0/22', '5/20',
       '1/22', '9/22', '1/16', '1/15', '2/15', '1/17', '1/18', '5/16',
       '1/19'], dtype=object)

In [104]:
iso_df[iso_df['date_proposed_raw'].str.endswith('1899', na=False)]

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,year_operational,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia
256,226,twin falls,ID,16,16083,31dec1899,NaN,IP,In Progress,SISR,...,NaN,NaN,NaN,NaN,NaN,Solar,100.00,1.0,[Idaho Power Co],[9191]
2525,2324,elmore,ID,16,16039,31dec1899,NaN,IP,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,Solar,240.00,3.0,"[Atlanta Power Co Inc, Idaho Power Co, Atlanta...","[967, 9191, 1738]"
2847,2600,lemhi,ID,16,16059,31dec1899,NaN,IP,Not Started,NaN,...,NaN,NaN,NaN,NaN,NaN,Hydro,0.75,3.0,"[Salmon River Electric Coop Inc, Idaho Power C...","[16565, 9191, 14354]"


In [114]:
test_df = iso_df[:10]

In [125]:
test_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,cleaned_year
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,Battery,725.0,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2...",NaN
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,Wind,4.5,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]",NaN
5,5,woodward,OK,40,40153,NaN,NaN,SPP,In Progress,DISIS STAGE,...,NaN,NaN,NaN,NaN,Battery,80.0,4.0,"[City of Mooreland, Town of Fort Supply, North...","[12892, 6618, 13807, 14063]",NaN
6,6,sherman,OR,41,41055,12/31/2021,NaN,BPA,In Progress,STUDY,...,NaN,NaN,NaN,NaN,Solar,800.0,4.0,"[PacifiCorp, Wasco Electric Coop, Inc, Columbi...","[14354, 20138, 4005, 1738]",NaN
7,8,benton,WA,53,53005,12/1/2019,NaN,BPA,In Progress,STUDY,...,NaN,NaN,NaN,NaN,Wind,250.0,6.0,"[PUD No 1 of Klickitat County, City of Richlan...","[10393, 15979, 1579, 1625, 14354, 1738]",NaN
8,9,coles,IL,17,17029,7/1/2022,NaN,MISO,In Progress,PHASE 1,...,NaN,NaN,NaN,NaN,Solar,99.0,4.0,"[Edgar Electric Co-op, Assn, Coles-Moultrie El...","[5531, 3931, 3253, 56697]",NaN
9,10,jerome,ID,16,16053,01dec2022,NaN,IP,In Progress,FeSR,...,NaN,NaN,NaN,NaN,Battery,NaN,1.0,[Idaho Power Co],[9191],NaN


### Initial basic cleaning to extract year - update: don't need this

In [134]:
import numpy as np
years = tuple([str(s) for s in np.arange(2000, 2051)])

In [140]:
iso_df.loc[iso_df['date_proposed_raw'].str.endswith(years, na=False), 'cleaned_year'] = iso_df['date_proposed_raw'].str[-4:]
iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,cleaned_year
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",2023
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",2023
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",2023
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,Battery,725.0000,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2...",2024
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,Wind,4.5000,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]",2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,NaN,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,NaN,Offshore Wind,1200.0000,NaN,NaN,NaN,2022
6350,5285,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,Solar,6.0000,NaN,NaN,NaN,NaN
6351,5352,NaN,NaN,NaN,NaN,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,NaN,Battery,0.0000,NaN,NaN,NaN,2024
6352,5378,NaN,NaN,NaN,NaN,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,NaN,Solar,0.0055,NaN,NaN,NaN,NaN


In [127]:
yrs = ('2000', '2002', '2003')
s = "572002"
s.endswith(yrs)

True

In [135]:
test_df['date_proposed_raw'].str.endswith(years, na=False)

0     True
1     True
2     True
3     True
4     True
5    False
6     True
7     True
8     True
9     True
Name: date_proposed_raw, dtype: bool

In [139]:
test_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,date_withdrawn_raw,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,cleaned_year
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,NaN,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",2023
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,NaN,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",2023
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,NaN,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",2023
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,NaN,Battery,725.0,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2...",2024
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,NaN,Wind,4.5,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]",2021
5,5,woodward,OK,40,40153,NaN,NaN,SPP,In Progress,DISIS STAGE,...,NaN,NaN,NaN,NaN,Battery,80.0,4.0,"[City of Mooreland, Town of Fort Supply, North...","[12892, 6618, 13807, 14063]",NaN
6,6,sherman,OR,41,41055,12/31/2021,NaN,BPA,In Progress,STUDY,...,NaN,NaN,NaN,NaN,Solar,800.0,4.0,"[PacifiCorp, Wasco Electric Coop, Inc, Columbi...","[14354, 20138, 4005, 1738]",2021
7,8,benton,WA,53,53005,12/1/2019,NaN,BPA,In Progress,STUDY,...,NaN,NaN,NaN,NaN,Wind,250.0,6.0,"[PUD No 1 of Klickitat County, City of Richlan...","[10393, 15979, 1579, 1625, 14354, 1738]",2019
8,9,coles,IL,17,17029,7/1/2022,NaN,MISO,In Progress,PHASE 1,...,NaN,NaN,NaN,NaN,Solar,99.0,4.0,"[Edgar Electric Co-op, Assn, Coles-Moultrie El...","[5531, 3931, 3253, 56697]",2022
9,10,jerome,ID,16,16053,01dec2022,NaN,IP,In Progress,FeSR,...,NaN,NaN,NaN,NaN,Battery,NaN,1.0,[Idaho Power Co],[9191],2022


### Join in utliity data based on year and county

In [188]:
iso_df['year_proposed'] = pd.to_datetime(iso_df['date_proposed']).dt.year

In [189]:
iso_df['year_proposed']

0       2023.0
1       2023.0
2       2023.0
3       2024.0
4       2021.0
         ...  
6349    2022.0
6350       NaN
6351    2024.0
6352       NaN
6353       NaN
Name: year_proposed, Length: 6354, dtype: float64

In [178]:
serv_terr_df['year'] = serv_terr_df['report_date'].dt.year.astype(float)

In [179]:
serv_terr_df

,county,short_form,state,utility_id_eia,utility_name_eia,report_date,state_id_fips,county_id_fips,year
0,Carroll,<NA>,IA,11581,City of Manning,2001-01-01,19,19027,2001.0
1,Wright,<NA>,MO,11585,City of Mansfield,2001-01-01,29,29229,2001.0
2,Bristol,<NA>,MA,11586,Town of Mansfield,2001-01-01,25,25005,2001.0
3,Newton,<NA>,GA,11587,City of Mansfield,2001-01-01,13,13217,2001.0
4,Sanpete,<NA>,UT,11588,City of Manti,2001-01-01,49,49039,2001.0
...,...,...,...,...,...,...,...,...,...
235986,Neosho,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20133,2020.0
235987,Osage,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20139,2020.0
235988,Wabaunsee,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20197,2020.0
235989,Wilson,<NA>,KS,64196,"4 Rivers Electric Cooperative, Inc.",2020-01-01,20,20205,2020.0


In [197]:
ids_per_county_year = serv_terr_df.groupby(['county_id_fips', 'year'])['utility_id_eia'].unique()
ids_per_county_year = ids_per_county_year.rename('utility_ids_in_county_year')
ids_per_county_year

county_id_fips  year  
01001           2001.0           [195, 3222]
                2002.0           [195, 3222]
                2003.0           [195, 3222]
                2004.0           [195, 3222]
                2005.0    [195, 3222, 30517]
                                 ...        
72153           2012.0               [15497]
                2013.0               [15497]
                2014.0               [15497]
                2015.0               [15497]
                2016.0               [15497]
Name: utility_ids_in_county_year, Length: 63789, dtype: object

In [198]:
names_per_county_year = serv_terr_df.groupby(['county_id_fips', 'year'])['utility_name_eia'].unique()
names_per_county_year = names_per_county_year.rename('utility_names_in_county_year')
names_per_county_year

county_id_fips  year  
01001           2001.0    [Alabama Power Co, Central Alabama Electric Coop]
                2002.0    [Alabama Power Co, Central Alabama Electric Coop]
                2003.0    [Alabama Power Co, Central Alabama Electric Coop]
                2004.0    [Alabama Power Co, Central Alabama Electric Coop]
                2005.0    [Alabama Power Co, Central Alabama Electric Co...
                                                ...                        
72153           2012.0                 [Puerto Rico Electric Pwr Authority]
                2013.0                 [Puerto Rico Electric Pwr Authority]
                2014.0                 [Puerto Rico Electric Pwr Authority]
                2015.0                 [Puerto Rico Electric Pwr Authority]
                2016.0                 [Puerto Rico Electric Pwr Authority]
Name: utility_names_in_county_year, Length: 63789, dtype: object

In [218]:
iso_df = iso_df.merge(ids_per_county_year, how='left', left_on=['county_id_fips', 'year_proposed'], right_index=True)

In [219]:
iso_df = iso_df.merge(names_per_county_year, how='left', left_on=['county_id_fips', 'year_proposed'], right_index=True)
iso_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year,utility_names_in_county_year
0,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN
1,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN
2,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,NaN,NaN,NaN,Battery,500.0000,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN
3,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,NaN,NaN,NaN,Battery,725.0000,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2...",NaN,NaN
4,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,NaN,NaN,NaN,Wind,4.5000,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,5215,NaN,NaN,NaN,nan,23jun2022,NaN,ISO-NE,Not Started,NaN,...,NaN,NaN,NaN,Offshore Wind,1200.0000,NaN,NaN,NaN,NaN,NaN
6350,5285,NaN,NaN,NaN,nan,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,Solar,6.0000,NaN,NaN,NaN,NaN,NaN
6351,5352,NaN,NaN,NaN,nan,12/31/2024,NaN,PJM,Not Started,Not Started,...,NaN,NaN,NaN,Battery,0.0000,NaN,NaN,NaN,NaN,NaN
6352,5378,NaN,NaN,NaN,nan,NaN,NaN,Dominion,unknown,NaN,...,NaN,NaN,NaN,Solar,0.0055,NaN,NaN,NaN,NaN,NaN


### Merge in utility data based on closest year

In [251]:
iso_df = iso_df.sort_values(by=['year_proposed'])

In [235]:
ids_per_county_year = ids_per_county_year.reset_index()
ids_per_county_year = ids_per_county_year.sort_values('year')

In [250]:
ids_per_county_year

,county_id_fips,year,utility_ids_in_county_year
0,01001,2001.0,"[195, 3222]"
15278,18141,2001.0,"[12674, 13423, 20021, 12377, 11767, 9999, 1375..."
54626,48429,2001.0,"[40051, 4295, 6611, 19490, 44372]"
15258,18139,2001.0,"[10407, 8000, 4960, 8466, 17038, 15470]"
15238,18137,2001.0,"[17599, 4960, 15470]"
...,...,...,...
53945,48359,2020.0,"[4939, 6198, 16063, 17718]"
34301,31131,2020.0,"[13334, 13337, 14127, 18408, 18428]"
46985,45077,2020.0,"[1890, 5416, 6709]"
27736,27153,2020.0,"[4577, 10618, 12647, 13781, 14232, 16368, 1798..."


In [256]:
names_per_county_year = names_per_county_year.reset_index()
names_per_county_year = names_per_county_year.sort_values('year')

In [253]:
iso_df_no_nan = iso_df[~((iso_df['year_proposed'].isnull()) | (iso_df['county_id_fips'] == 'nan'))]
iso_df_nan = iso_df[(iso_df['year_proposed'].isnull()) | (iso_df['county_id_fips'] == 'nan')]
iso_df_no_nan

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,withdrawl_reason,year_withdrawn,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year,utility_names_in_county_year
1283,1154,johnson,IA,19,19103,1/1/2006,NaN,MISO,In Progress,PHASE 2,...,NaN,NaN,NaN,Other,2.0,8.0,"[T I P Rural Electric Coop, Linn County Rural ...","[18446, 11053, 9162, 5605, 5588, 12341, 9417, ...","[5588, 5605, 6168, 9417, 11053, 12341, 18446]","[East-Central Iowa Rural Elec Coop, Eastern Io..."
887,805,randolph,MO,29,29175,5/1/2010,NaN,AEC,IA Executed,Upgrade Approved,...,NaN,NaN,NaN,Coal,1.7,5.0,"[Howard Electric Cooperative, Boone Electric C...","[8911, 2001, 11463, 19436, 10000]","[2001, 8911, 10000, 11463, 19436]","[Boone Electric Coop, Howard Electric Coop, Ka..."
6141,5576,bingham,ID,16,16011,7/1/2010,"Cedar Creek Wind, LLC",PacifiCorp,IA Executed,IA Executed,...,NaN,NaN,NaN,Wind,151.8,3.0,"[Idaho Power Co, PacifiCorp, Bonneville Power ...","[9191, 14354, 1738]","[9191, 14354]","[Idaho Power Co, PacifiCorp]"
2077,1908,wasco,OR,41,41065,8/30/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,NaN,NaN,NaN,Wind,201.0,5.0,"[Northern Wasco County PUD, Wasco Electric Coo...","[13788, 20138, 14354, 3240, 1738]","[3240, 13788, 14354, 20138]","[Central Electric Coop Inc, Northern Wasco Cou..."
3487,3176,gilliam,OR,41,41021,9/1/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,NaN,NaN,NaN,Wind,202.0,4.0,"[Columbia Basin Elec Coop, Inc, Wasco Electric...","[4005, 20138, 14354, 1738]","[4005, 14354, 20138]","[Columbia Basin Elec Cooperative, Inc, PacifiC..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,1111,nassau,NY,36,36059,11/2027,Equinor Wind US LLC,NYISO,In Progress,FES,...,NaN,NaN,NaN,Offshore Wind,1300.0,4.0,"[Village of Freeport, Long Island Power Author...","[6775, 11171, 16217, 10304]",NaN,NaN
1003,898,queens,NY,36,36081,11/2028,Equinor Wind US LLC,NYISO,In Progress,SRIS,...,NaN,NaN,NaN,Offshore Wind,1300.0,4.0,"[Long Island Power Authority, Consolidated Edi...","[11171, 4226, 13726, 10304]",NaN,NaN
5893,5360,miami-dade,FL,12,12086,01jun2028,NaN,FPL,In Progress,Yes,...,NaN,NaN,NaN,Nuclear,1300.0,2.0,"[City of Homestead, Florida Power & Light Comp...","[8795, 6452]",NaN,NaN
4044,3688,walla walla,WA,53,53071,7/25/2031,NaN,BPA,In Progress,STUDY,...,NaN,NaN,NaN,Hydro,210.0,4.0,"[Columbia Rural Elec Assn, Inc, Inland Power &...","[4041, 8699, 14354, 1738]",NaN,NaN


In [255]:
#tol = pd.Timedelta('5 year')
iso_merged_df = pd.merge_asof(left=iso_df_no_nan, right=ids_per_county_year, left_on='year_proposed', right_on='year', by='county_id_fips', direction='nearest', suffixes=('_exact', '_nearest'))
iso_merged_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,date_withdrawn,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year_exact,utility_names_in_county_year,year,utility_ids_in_county_year_nearest
0,1154,johnson,IA,19,19103,1/1/2006,NaN,MISO,In Progress,PHASE 2,...,NaN,Other,2.0,8.0,"[T I P Rural Electric Coop, Linn County Rural ...","[18446, 11053, 9162, 5605, 5588, 12341, 9417, ...","[5588, 5605, 6168, 9417, 11053, 12341, 18446]","[East-Central Iowa Rural Elec Coop, Eastern Io...",2006.0,"[5588, 5605, 6168, 9417, 11053, 12341, 18446]"
1,805,randolph,MO,29,29175,5/1/2010,NaN,AEC,IA Executed,Upgrade Approved,...,NaN,Coal,1.7,5.0,"[Howard Electric Cooperative, Boone Electric C...","[8911, 2001, 11463, 19436, 10000]","[2001, 8911, 10000, 11463, 19436]","[Boone Electric Coop, Howard Electric Coop, Ka...",2010.0,"[2001, 8911, 10000, 11463, 19436]"
2,5576,bingham,ID,16,16011,7/1/2010,"Cedar Creek Wind, LLC",PacifiCorp,IA Executed,IA Executed,...,NaN,Wind,151.8,3.0,"[Idaho Power Co, PacifiCorp, Bonneville Power ...","[9191, 14354, 1738]","[9191, 14354]","[Idaho Power Co, PacifiCorp]",2010.0,"[9191, 14354]"
3,1908,wasco,OR,41,41065,8/30/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,NaN,Wind,201.0,5.0,"[Northern Wasco County PUD, Wasco Electric Coo...","[13788, 20138, 14354, 3240, 1738]","[3240, 13788, 14354, 20138]","[Central Electric Coop Inc, Northern Wasco Cou...",2010.0,"[3240, 13788, 14354, 20138]"
4,3176,gilliam,OR,41,41021,9/1/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,NaN,Wind,202.0,4.0,"[Columbia Basin Elec Coop, Inc, Wasco Electric...","[4005, 20138, 14354, 1738]","[4005, 14354, 20138]","[Columbia Basin Elec Cooperative, Inc, PacifiC...",2010.0,"[4005, 14354, 20138]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,1111,nassau,NY,36,36059,11/2027,Equinor Wind US LLC,NYISO,In Progress,FES,...,NaN,Offshore Wind,1300.0,4.0,"[Village of Freeport, Long Island Power Author...","[6775, 11171, 16217, 10304]",NaN,NaN,2020.0,"[6775, 11171, 16217]"
5556,898,queens,NY,36,36081,11/2028,Equinor Wind US LLC,NYISO,In Progress,SRIS,...,NaN,Offshore Wind,1300.0,4.0,"[Long Island Power Authority, Consolidated Edi...","[11171, 4226, 13726, 10304]",NaN,NaN,2020.0,"[4226, 11171, 13726]"
5557,5360,miami-dade,FL,12,12086,01jun2028,NaN,FPL,In Progress,Yes,...,NaN,Nuclear,1300.0,2.0,"[City of Homestead, Florida Power & Light Comp...","[8795, 6452]",NaN,NaN,2020.0,"[6452, 8795]"
5558,3688,walla walla,WA,53,53071,7/25/2031,NaN,BPA,In Progress,STUDY,...,NaN,Hydro,210.0,4.0,"[Columbia Rural Elec Assn, Inc, Inland Power &...","[4041, 8699, 14354, 1738]",NaN,NaN,2020.0,"[1738, 4041, 8699, 14354]"


In [257]:
iso_merged_df = pd.merge_asof(left=iso_merged_df, right=names_per_county_year, left_on='year_proposed', right_on='year', by='county_id_fips', direction='nearest', suffixes=('_exact', '_nearest'))
iso_merged_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year_exact,utility_names_in_county_year_exact,year_exact,utility_ids_in_county_year_nearest,year_nearest,utility_names_in_county_year_nearest
0,1154,johnson,IA,19,19103,1/1/2006,NaN,MISO,In Progress,PHASE 2,...,2.0,8.0,"[T I P Rural Electric Coop, Linn County Rural ...","[18446, 11053, 9162, 5605, 5588, 12341, 9417, ...","[5588, 5605, 6168, 9417, 11053, 12341, 18446]","[East-Central Iowa Rural Elec Coop, Eastern Io...",2006.0,"[5588, 5605, 6168, 9417, 11053, 12341, 18446]",2006.0,"[East-Central Iowa Rural Elec Coop, Eastern Io..."
1,805,randolph,MO,29,29175,5/1/2010,NaN,AEC,IA Executed,Upgrade Approved,...,1.7,5.0,"[Howard Electric Cooperative, Boone Electric C...","[8911, 2001, 11463, 19436, 10000]","[2001, 8911, 10000, 11463, 19436]","[Boone Electric Coop, Howard Electric Coop, Ka...",2010.0,"[2001, 8911, 10000, 11463, 19436]",2010.0,"[Boone Electric Coop, Howard Electric Coop, Ka..."
2,5576,bingham,ID,16,16011,7/1/2010,"Cedar Creek Wind, LLC",PacifiCorp,IA Executed,IA Executed,...,151.8,3.0,"[Idaho Power Co, PacifiCorp, Bonneville Power ...","[9191, 14354, 1738]","[9191, 14354]","[Idaho Power Co, PacifiCorp]",2010.0,"[9191, 14354]",2010.0,"[Idaho Power Co, PacifiCorp]"
3,1908,wasco,OR,41,41065,8/30/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,201.0,5.0,"[Northern Wasco County PUD, Wasco Electric Coo...","[13788, 20138, 14354, 3240, 1738]","[3240, 13788, 14354, 20138]","[Central Electric Coop Inc, Northern Wasco Cou...",2010.0,"[3240, 13788, 14354, 20138]",2010.0,"[Central Electric Coop Inc, Northern Wasco Cou..."
4,3176,gilliam,OR,41,41021,9/1/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,202.0,4.0,"[Columbia Basin Elec Coop, Inc, Wasco Electric...","[4005, 20138, 14354, 1738]","[4005, 14354, 20138]","[Columbia Basin Elec Cooperative, Inc, PacifiC...",2010.0,"[4005, 14354, 20138]",2010.0,"[Columbia Basin Elec Cooperative, Inc, PacifiC..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,1111,nassau,NY,36,36059,11/2027,Equinor Wind US LLC,NYISO,In Progress,FES,...,1300.0,4.0,"[Village of Freeport, Long Island Power Author...","[6775, 11171, 16217, 10304]",NaN,NaN,2020.0,"[6775, 11171, 16217]",2020.0,"[Village of Freeport - (NY), Long Island Power..."
5556,898,queens,NY,36,36081,11/2028,Equinor Wind US LLC,NYISO,In Progress,SRIS,...,1300.0,4.0,"[Long Island Power Authority, Consolidated Edi...","[11171, 4226, 13726, 10304]",NaN,NaN,2020.0,"[4226, 11171, 13726]",2020.0,"[Consolidated Edison Co-NY Inc, Long Island Po..."
5557,5360,miami-dade,FL,12,12086,01jun2028,NaN,FPL,In Progress,Yes,...,1300.0,2.0,"[City of Homestead, Florida Power & Light Comp...","[8795, 6452]",NaN,NaN,2020.0,"[6452, 8795]",2020.0,"[Florida Power & Light Co, City of Homestead -..."
5558,3688,walla walla,WA,53,53071,7/25/2031,NaN,BPA,In Progress,STUDY,...,210.0,4.0,"[Columbia Rural Elec Assn, Inc, Inland Power &...","[4041, 8699, 14354, 1738]",NaN,NaN,2020.0,"[1738, 4041, 8699, 14354]",2020.0,"[Bonneville Power Administration, Columbia Rur..."


In [265]:
iso_merged_df = iso_merged_df.drop('year_exact', axis=1)
iso_merged_df = iso_merged_df.rename(columns={'year_nearest': 'utility_nearest_year'})
s = iso_merged_df.pop('utility_nearest_year')
iso_merged_df = pd.concat([iso_merged_df, s], 1)
iso_merged_df

/var/folders/cs/pq2nw6_x49l0mx3k8h6lps440000gn/T/ipykernel_80481/3350899064.py:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  iso_merged_df = pd.concat([iso_merged_df, s], 1)


,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year_exact,utility_names_in_county_year_exact,utility_ids_in_county_year_nearest,utility_names_in_county_year_nearest,utility_nearest_year
0,1154,johnson,IA,19,19103,1/1/2006,NaN,MISO,In Progress,PHASE 2,...,Other,2.0,8.0,"[T I P Rural Electric Coop, Linn County Rural ...","[18446, 11053, 9162, 5605, 5588, 12341, 9417, ...","[5588, 5605, 6168, 9417, 11053, 12341, 18446]","[East-Central Iowa Rural Elec Coop, Eastern Io...","[5588, 5605, 6168, 9417, 11053, 12341, 18446]","[East-Central Iowa Rural Elec Coop, Eastern Io...",2006.0
1,805,randolph,MO,29,29175,5/1/2010,NaN,AEC,IA Executed,Upgrade Approved,...,Coal,1.7,5.0,"[Howard Electric Cooperative, Boone Electric C...","[8911, 2001, 11463, 19436, 10000]","[2001, 8911, 10000, 11463, 19436]","[Boone Electric Coop, Howard Electric Coop, Ka...","[2001, 8911, 10000, 11463, 19436]","[Boone Electric Coop, Howard Electric Coop, Ka...",2010.0
2,5576,bingham,ID,16,16011,7/1/2010,"Cedar Creek Wind, LLC",PacifiCorp,IA Executed,IA Executed,...,Wind,151.8,3.0,"[Idaho Power Co, PacifiCorp, Bonneville Power ...","[9191, 14354, 1738]","[9191, 14354]","[Idaho Power Co, PacifiCorp]","[9191, 14354]","[Idaho Power Co, PacifiCorp]",2010.0
3,1908,wasco,OR,41,41065,8/30/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,Wind,201.0,5.0,"[Northern Wasco County PUD, Wasco Electric Coo...","[13788, 20138, 14354, 3240, 1738]","[3240, 13788, 14354, 20138]","[Central Electric Coop Inc, Northern Wasco Cou...","[3240, 13788, 14354, 20138]","[Central Electric Coop Inc, Northern Wasco Cou...",2010.0
4,3176,gilliam,OR,41,41021,9/1/2010,NaN,BPA,IA Executed,STUDY COMPLETED,...,Wind,202.0,4.0,"[Columbia Basin Elec Coop, Inc, Wasco Electric...","[4005, 20138, 14354, 1738]","[4005, 14354, 20138]","[Columbia Basin Elec Cooperative, Inc, PacifiC...","[4005, 14354, 20138]","[Columbia Basin Elec Cooperative, Inc, PacifiC...",2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5555,1111,nassau,NY,36,36059,11/2027,Equinor Wind US LLC,NYISO,In Progress,FES,...,Offshore Wind,1300.0,4.0,"[Village of Freeport, Long Island Power Author...","[6775, 11171, 16217, 10304]",NaN,NaN,"[6775, 11171, 16217]","[Village of Freeport - (NY), Long Island Power...",2020.0
5556,898,queens,NY,36,36081,11/2028,Equinor Wind US LLC,NYISO,In Progress,SRIS,...,Offshore Wind,1300.0,4.0,"[Long Island Power Authority, Consolidated Edi...","[11171, 4226, 13726, 10304]",NaN,NaN,"[4226, 11171, 13726]","[Consolidated Edison Co-NY Inc, Long Island Po...",2020.0
5557,5360,miami-dade,FL,12,12086,01jun2028,NaN,FPL,In Progress,Yes,...,Nuclear,1300.0,2.0,"[City of Homestead, Florida Power & Light Comp...","[8795, 6452]",NaN,NaN,"[6452, 8795]","[Florida Power & Light Co, City of Homestead -...",2020.0
5558,3688,walla walla,WA,53,53071,7/25/2031,NaN,BPA,In Progress,STUDY,...,Hydro,210.0,4.0,"[Columbia Rural Elec Assn, Inc, Inland Power &...","[4041, 8699, 14354, 1738]",NaN,NaN,"[1738, 4041, 8699, 14354]","[Bonneville Power Administration, Columbia Rur...",2020.0


In [268]:
iso_merged_df = pd.concat([iso_merged_df, iso_df_nan.drop(['utility_ids_in_county_year', 'utility_names_in_county_year'], axis=1)])
iso_merged_df = iso_merged_df.sort_values('project_id')
iso_merged_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year_exact,utility_names_in_county_year_exact,utility_ids_in_county_year_nearest,utility_names_in_county_year_nearest,utility_nearest_year,utility_ids_in_county_year,utility_names_in_county_year
3965,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN,"[12463, 12825, 20997, 21513, 27000]","[Mid-Yellowstone Elec Coop, Inc, NorthWestern ...",2020.0,NaN,NaN
4660,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN,"[12463, 12825, 20997, 21513, 27000]","[Mid-Yellowstone Elec Coop, Inc, NorthWestern ...",2020.0,NaN,NaN
4664,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN,"[12463, 12825, 20997, 21513, 27000]","[Mid-Yellowstone Elec Coop, Inc, NorthWestern ...",2020.0,NaN,NaN
5042,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2...",NaN,NaN,"[694, 1192, 4390, 9216, 16088, 17609, 27000, 5...","[Anza Electric Coop Inc, City of Banning - (CA...",2020.0,NaN,NaN
1022,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]",NaN,NaN,"[296, 3226, 3478, 10012, 14063]","[Alfalfa Electric Coop, Inc, Central Rural Ele...",2020.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,5634,tooele,UT,49,49045,12/31/2019,"Ophir Canyon Solar, LLC",PacifiCorp,IA Executed,IA Executed,...,3.0,"[Wells Rural Electric Co, Mt Wheeler Power, In...","[20332, 13073, 14354]","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...",2019.0,NaN,NaN
373,5635,tooele,UT,49,49045,12/31/2019,"Ophir Canyon Solar, LLC",PacifiCorp,IA Executed,IA Executed,...,3.0,"[Wells Rural Electric Co, Mt Wheeler Power, In...","[20332, 13073, 14354]","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...",2019.0,NaN,NaN
962,5636,jefferson,OR,41,41031,1/1/2021,Deschutes Valley Water District,PacifiCorp,IA Executed,IA Executed,...,4.0,"[Wasco Electric Coop, Inc, Central Electric Co...","[20138, 3240, 14354, 1738]",NaN,NaN,"[1738, 3240, 14354, 20138]","[Bonneville Power Administration, Central Elec...",2020.0,NaN,NaN
142,5637,shasta,CA,06,06089,12/31/2018,Slate Creek Hydro Associates LP,PacifiCorp,IA Executed,IA Executed,...,5.0,"[City of Redding, City of Shasta City, PacifiC...","[15783, 17896, 14354, 14328, 27000]","[14328, 14354, 15783, 17896, 27000]","[Pacific Gas & Electric Co., PacifiCorp, City ...","[14328, 14354, 15783, 17896, 27000]","[Pacific Gas & Electric Co., PacifiCorp, City ...",2018.0,NaN,NaN


In [270]:
iso_merged_df

,project_id,county,state,state_id_fips,county_id_fips,date_proposed_raw,developer,entity,interconnection_status_lbnl,interconnection_status_raw,...,resource,capacity_mw,n_utilities_in_county,utilities_in_county_eia,utility_ids_in_county_eia,utility_ids_in_county_year_exact,utility_names_in_county_year_exact,utility_ids_in_county_year_nearest,utility_names_in_county_year_nearest,utility_nearest_year
3965,0,yellowstone,MT,30,30111,12/31/2023,NaN,NWE,In Progress,Active,...,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN,"[12463, 12825, 20997, 21513, 27000]","[Mid-Yellowstone Elec Coop, Inc, NorthWestern ...",2020.0
4660,1,yellowstone,MT,30,30111,12/31/2023,NaN,PacifiCorp,In Progress,In Progress,...,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN,"[12463, 12825, 20997, 21513, 27000]","[Mid-Yellowstone Elec Coop, Inc, NorthWestern ...",2020.0
4664,2,yellowstone,MT,30,30111,12/31/2023,NaN,Avista,Not Started,New,...,Battery,500.0,5.0,"[Yellowstone Valley Elec Co-op, Mid-Yellowston...","[20997, 12463, 21513, 12825, 27000]",NaN,NaN,"[12463, 12825, 20997, 21513, 27000]","[Mid-Yellowstone Elec Coop, Inc, NorthWestern ...",2020.0
5042,3,riverside,CA,06,06065,11/15/2024,NaN,CAISO,Not Started,NaN,...,Battery,725.0,8.0,"[Imperial Irrigation District, City of Riversi...","[9216, 16088, 694, 1192, 17609, 4390, 55787, 2...",NaN,NaN,"[694, 1192, 4390, 9216, 16088, 17609, 27000, 5...","[Anza Electric Coop Inc, City of Banning - (CA...",2020.0
1022,4,garfield,OK,40,40047,12/31/2021,NaN,SPP,IA Executed,IA FULLY EXECUTED/ON SCHEDULE,...,Wind,4.5,5.0,"[Kay Electric Cooperative, Central Rural Elect...","[10012, 3226, 296, 3478, 14063]",NaN,NaN,"[296, 3226, 3478, 10012, 14063]","[Alfalfa Electric Coop, Inc, Central Rural Ele...",2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,5634,tooele,UT,49,49045,12/31/2019,"Ophir Canyon Solar, LLC",PacifiCorp,IA Executed,IA Executed,...,Solar,10.0,3.0,"[Wells Rural Electric Co, Mt Wheeler Power, In...","[20332, 13073, 14354]","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...",2019.0
373,5635,tooele,UT,49,49045,12/31/2019,"Ophir Canyon Solar, LLC",PacifiCorp,IA Executed,IA Executed,...,Solar,31.5,3.0,"[Wells Rural Electric Co, Mt Wheeler Power, In...","[20332, 13073, 14354]","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...","[13073, 14354, 20332]","[Mt Wheeler Power, Inc, PacifiCorp, Wells Rura...",2019.0
962,5636,jefferson,OR,41,41031,1/1/2021,Deschutes Valley Water District,PacifiCorp,IA Executed,IA Executed,...,Hydro,4.3,4.0,"[Wasco Electric Coop, Inc, Central Electric Co...","[20138, 3240, 14354, 1738]",NaN,NaN,"[1738, 3240, 14354, 20138]","[Bonneville Power Administration, Central Elec...",2020.0
142,5637,shasta,CA,06,06089,12/31/2018,Slate Creek Hydro Associates LP,PacifiCorp,IA Executed,IA Executed,...,Hydro,4.2,5.0,"[City of Redding, City of Shasta City, PacifiC...","[15783, 17896, 14354, 14328, 27000]","[14328, 14354, 15783, 17896, 27000]","[Pacific Gas & Electric Co., PacifiCorp, City ...","[14328, 14354, 15783, 17896, 27000]","[Pacific Gas & Electric Co., PacifiCorp, City ...",2018.0


In [271]:
iso_merged_df.to_csv('active_iso_projects_with_utility.csv', index=False)

In [272]:
iso_merged_df.columns

Index(['project_id', 'county', 'state', 'state_id_fips', 'county_id_fips',
       'date_proposed_raw', 'developer', 'entity',
       'interconnection_status_lbnl', 'interconnection_status_raw',
       'point_of_interconnection', 'project_name', 'queue_date', 'queue_id',
       'queue_status', 'queue_year', 'region', 'resource_type_lbnl', 'utility',
       'year_proposed', 'date_proposed', 'date_operational', 'days_in_queue',
       'queue_date_raw', 'year_operational', 'date_withdrawn_raw',
       'withdrawl_reason', 'year_withdrawn', 'date_withdrawn', 'resource',
       'capacity_mw', 'n_utilities_in_county', 'utilities_in_county_eia',
       'utility_ids_in_county_eia', 'utility_ids_in_county_year_exact',
       'utility_names_in_county_year_exact',
       'utility_ids_in_county_year_nearest',
       'utility_names_in_county_year_nearest', 'utility_nearest_year'],
      dtype='object')